# Learn how to make WOMBAT intermediates

The raw GEOSCHEM and observational data is processed into two types of file:
- monthly_fluxes.nc
- combined_mf.nc

The netcdf files are processed into five intermediate files needed to run WOMBAT, which are:
- control-emissions.fst
- perturbations.fst
- control-mole-fraction.fst
- sensitivities.fst
- observations.fst

These can be generated however you want, but need to match the WOMBAT format.

These can then be combined into a "measurement model" and "process model", which are joined together into a "real model" and used with the WOMBAT module to do the inversion.

This tutorial points you to the scripts to create these intermediates, and also shows you what these intermediates look like. It then uses those intermediates to create, and examine the contents of, the models. 




## Setup notebook

The WOMBAT module code itself is written in R. A mixture of python and R is used to generate the intermediates used in WOMBAT, along with bash scripts that join python and R workflows.

Import required python modules:

In [1]:
import configparser
import os
from pathlib import Path
import subprocess

import xarray as xr

Enable R in this notebook:

In [3]:
%load_ext rpy2.ipython

Import required R libraries:

In [4]:
%%R

library(fst)
library(ini)

R[write to console]: fst package v0.9.4

R[write to console]: (OpenMP detected, using 48 threads)



I've tried to use a config file that contains all the paths and constants that might need to be changed. 

Read config file into python:

In [5]:
config = configparser.ConfigParser()
config.read('../../config.ini')

['../../config.ini']

Read config file into R:

In [6]:
%%R

config <- read.ini("../../config.ini")

## 1. Extract the emissions from the GEOSCHEM runs

Run the process_geos_ems.py script in the cell below, this takes a couple of minutes.

This script takes three arguments to allow us to make the cutdown version and the full version in the same script. The first argument is the first year of data to be read in and the second argument is the last year to be read in. For this tutorial, only one year of data is read in. The third argument is what to save the emissions output as, here it is "monthly_fluxes_tutorial.nc", in the full version it is "monthly_fluxes.nc".

This saves the monthly fluxes in the geoschem output folders, one for each of the base run and the perturbation runs.

In [7]:
%run ../intermediates/process_geos_ems.py 2010 2010 "monthly_fluxes_tutorial.nc" 

Loading data from 2010 - 2010
Saving data to monthly_fluxes_tutorial.nc
/user/work/as16992/geoschem/N2O/output/201001
[PosixPath('/user/work/as16992/geoschem/N2O/output/201001/HEMCO_diagnostics.201001010000.nc'), PosixPath('/user/work/as16992/geoschem/N2O/output/201001/HEMCO_diagnostics.201002010000.nc'), PosixPath('/user/work/as16992/geoschem/N2O/output/201001/HEMCO_diagnostics.201003010000.nc'), PosixPath('/user/work/as16992/geoschem/N2O/output/201001/HEMCO_diagnostics.201004010000.nc'), PosixPath('/user/work/as16992/geoschem/N2O/output/201001/HEMCO_diagnostics.201005010000.nc'), PosixPath('/user/work/as16992/geoschem/N2O/output/201001/HEMCO_diagnostics.201006010000.nc'), PosixPath('/user/work/as16992/geoschem/N2O/output/201001/HEMCO_diagnostics.201007010000.nc'), PosixPath('/user/work/as16992/geoschem/N2O/output/201001/HEMCO_diagnostics.201008010000.nc'), PosixPath('/user/work/as16992/geoschem/N2O/output/201001/HEMCO_diagnostics.201009010000.nc'), PosixPath('/user/work/as16992/geosc

Have a look at one of the monthly_fluxes_tutorial.nc (this one is for the base run):

In [8]:
with xr.open_dataset(Path(config["paths"]["geos_out"]) / config["inversion_constants"]["case"] / "monthly_fluxes_tutorial.nc") as load:
    xr_data = load.load()

print(xr_data)

<xarray.Dataset>
Dimensions:          (longitude: 72, latitude: 46, time: 12, longitude_width: 72, latitude_height: 46)
Coordinates:
  * longitude        (longitude) float64 -180.0 -175.0 -170.0 ... 170.0 175.0
  * latitude         (latitude) float64 -89.0 -86.0 -82.0 ... 82.0 86.0 89.0
  * time             (time) datetime64[ns] 2010-01-01 2010-02-01 ... 2010-12-01
  * longitude_width  (longitude_width) float64 5.0 5.0 5.0 5.0 ... 5.0 5.0 5.0
  * latitude_height  (latitude_height) float64 2.0 4.0 4.0 4.0 ... 4.0 4.0 2.0
Data variables: (12/25)
    AREA             (time, latitude, longitude) float64 2.158e+09 ... 2.158e+09
    EMIS_CH4_R22     (time, latitude, longitude) float32 0.0 0.0 0.0 ... 0.0 0.0
    EMIS_CH4_R21     (time, latitude, longitude) float32 0.0 0.0 0.0 ... 0.0 0.0
    EMIS_CH4_R20     (time, latitude, longitude) float32 0.0 0.0 0.0 ... 0.0 0.0
    EMIS_CH4_R19     (time, latitude, longitude) float32 0.0 0.0 0.0 ... 0.0 0.0
    EMIS_CH4_R18     (time, latitude, longitu

The file contains the area of the GEOSCHEM grid cells (AREA), the emissions for each region as a separate tracer (EMIS_CH4_R??), as well as the total emissions: EMIS_CH4_TOTAL (all regions summed).

## 2. Make the control emissions intermediate

This turns the base run's monthly_fluxes_tutorial.nc file into the control emissions intermediate: control-emissions-tutorial.fst.

In [9]:
subprocess.call('Rscript ../intermediates/control-emissions.R  --flux-file "monthly_fluxes_tutorial.nc" --output "control-emissions-tutorial.fst"', shell = True)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

here() starts at /user/home/as16992/global_n2o_inversion
Warning message:
replacing previous import ‘Matrix::update’ by ‘stats::update’ when loading ‘wombat’ 


0

Look at the control emission intermediate:

In [10]:
%%R
control_ems <- fst::read_fst(sprintf("%s/%s", config$paths$geos_inte, "control-emissions-tutorial.fst"))

head(control_ems)

  month_start model_id       area  type flux_density longitude cell_width
1  2010-01-01        1 2157765258  land 1.010600e-14      -180          5
2  2010-01-01        1 2157765258 ocean 0.000000e+00      -180          5
3  2010-01-01        2 2157765258  land 9.989040e-15      -175          5
4  2010-01-01        2 2157765258 ocean 0.000000e+00      -175          5
5  2010-01-01        3 2157765258  land 9.757725e-15      -170          5
6  2010-01-01        3 2157765258 ocean 0.000000e+00      -170          5
  latitude cell_height
1      -89           2
2      -89           2
3      -89           2
4      -89           2
5      -89           2
6      -89           2


### Variables

- "month_start" identifies the time (emissions only change on a monthly basis)

- "model_id" identifies the grid cell and the month

- "area" is the area of the grid cell in m<sup>2</sup>

- "type" can be ocean or land - in the intermediates each grid cell has both an ocean and a land entry, but each grid cell is classified as either land or ocean according to the TRANSCOM regions

- "flux_density" is the gas emissions in kgm<sup>-2</sup>s<sup>-1</sup> for that grid cell. Half the values are zeros because grid cells are classified as either ocean or land  according to the TRANSCOM regions

- "longitude" is the longitudinal centre of the grid cell in degrees

- "cell_width" is the width of the grid cell in degrees

- "latitude" is the latitudinal centre of the grid cell in degrees

- "cell_height" is the height of the grid cell in degrees - my geoschem runs use half polar grid cells, so it's 2° at the poles and 4° elsewhere

### Ordering

Ordered by model_id then type

## 3. Make the perturbations intermediate

This script reads in the emissions (monthly_fluxes_tutorial.nc) from the perturbation runs, subtracts the emissions from the base run (control-emissions-tutorial.fst), and outputs the perturbations intermediate (perturbations_tutorial.fst).

In [11]:
subprocess.call('Rscript ../intermediates/perturbations.R --flux-file "monthly_fluxes_tutorial.nc" --control-ems "control-emissions-tutorial.fst" --output "perturbations_tutorial.fst"', shell = True)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

here() starts at /user/home/as16992/global_n2o_inversion

Attaching package: ‘lubridate’

The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union

Warning message:
replacing previous import ‘Matrix::update’ by ‘stats::update’ when loading ‘wombat’ 


[1] "/user/work/as16992/geoschem/N2O/output/201001/monthly_fluxes_tutorial.nc"
[1] 1
[1] 12
[1] "/user/work/as16992/geoschem/N2O/output/201001/monthly_fluxes_tutorial.nc"
[1] 1
[1] 12
[1] "/user/work/as16992/geoschem/N2O/output/201001/monthly_fluxes_tutorial.nc"
[1] 1
[1] 12
[1] "/user/work/as16992/geoschem/N2O/output/201001/monthly_fluxes_tutorial.nc"
[1] 1
[1] 12
[1] "/user/work/as16992/geoschem/N2O/output/201001/monthly_fluxes_tutorial.nc"
[1] 1
[1] 12
[1] "/user/work/as16992/geoschem/N2O/output/201001/monthly_fluxes_tutorial.nc"
[1] 1
[1] 12
[1] "/user/work/as16992/geoschem/N2O/output/201001/monthly_fluxes_tutorial.nc"
[1] 1
[1] 12
[1] "/user/work/as16992/geoschem/N2O/output/201001/monthly_fluxes_tutorial.nc"
[1] 1
[1] 12
[1] "/user/work/as16992/geoschem/N2O/output/201001/monthly_fluxes_tutorial.nc"
[1] 1
[1] 12
[1] "/user/work/as16992/geoschem/N2O/output/201001/monthly_fluxes_tutorial.nc"
[1] 1
[1] 12
[1] "/user/work/as16992/geoschem/N2O/output/201001/monthly_fluxes_tutorial.nc"
[

0

Look at the perturbation intermediate:

In [12]:
%%R

perturbations <- fst::read_fst(sprintf("%s/%s", config$paths$geos_inte, "perturbations_tutorial.fst"))

head(perturbations)


  region from_month_start  type model_id flux_density
1      0       2010-01-01  land        1 1.010600e-14
2      0       2010-01-01 ocean        1 0.000000e+00
3      0       2010-01-01  land        2 9.989040e-15
4      0       2010-01-01 ocean        2 0.000000e+00
5      0       2010-01-01  land        3 9.757725e-15
6      0       2010-01-01 ocean        3 0.000000e+00


### Variables

- "region" identifies the TRANSCOM region of the perturbation

- "from_month_start" identifies the time that the emissions were perturbed

- "type" can be ocean or land - in the intermediates each grid cell has both an ocean and a land entry, but each grid cell is classified as either land or ocean according to the TRANSCOM regions

- "model_id" identifies the grid cell and the month

- "flux_density" is the difference in gas emissions between the perturbed and the base run in kgm<sup>-2</sup>s<sup>-1</sup> for that grid cell (in my case, I just doubled the emissions in the perturbed run for the first month)

### Ordering

Ordered by from_month_start, then region, then type

## 4. Extract the mole fraction data from the GEOSCHEM runs

This is the slowest step, it'll take a while (14 mins for me). In my workflow I submit each of the 13 runs of the script in parallel so it runs faster. You have to run process_geos_output.py 13 times, 12 for each of the perturbation runs, and once for the base run. This tutorial only contains one year, so is relatively short, but the full run requires 133 runs (12*11 perturbations and the base run).

The first argument passed to the script is the index of the folder to find the geoschem mole fraction data in. A list of folders is established by list(sorted(Path(config["paths"]["geos_out"]).iterdir())) in python. On my system at least, the indices 0-11 contain the perturbation runs, and index 132 contains the base run. The second argument is the first year of data to process, and the third is the last year of data to process. The fourth argument is the name of the outputfile to be saved.

In [13]:
# run for the perturbed runs in 2010
for i in range(0, 12):
    print(i)
    subprocess.call(f'python ../intermediates/process_geos_output.py {i} 2010 2010 "combined_mf_tutorial.nc"', shell = True)

# make base run
%run ../intermediates/process_geos_output.py 132 2010 2010 "combined_mf_tutorial.nc"


0
0
/user/work/as16992/geoschem/N2O/output/201001
Reading in obs...
Reading in geos...
checking last file to be read in
/user/work/as16992/geoschem/N2O/output/201001/GEOSChem.ObsPack.20101231_0000z.nc4
Combining datasets...
Sorting out dims...
Making monthly mean...
Recombining sites...
Making new obspack_id...
1
1
/user/work/as16992/geoschem/N2O/output/201002
Reading in obs...
Reading in geos...
checking last file to be read in
/user/work/as16992/geoschem/N2O/output/201002/GEOSChem.ObsPack.20101231_0000z.nc4
Combining datasets...
Sorting out dims...
Making monthly mean...
Recombining sites...
Making new obspack_id...
2
2
/user/work/as16992/geoschem/N2O/output/201003
Reading in obs...
Reading in geos...
checking last file to be read in
/user/work/as16992/geoschem/N2O/output/201003/GEOSChem.ObsPack.20101231_0000z.nc4
Combining datasets...
Sorting out dims...
Making monthly mean...
Recombining sites...
Making new obspack_id...
3
3
/user/work/as16992/geoschem/N2O/output/201004
Reading in 

Look at a combined_mf_tutorial.nc file (this one's for the base run):

In [14]:
with xr.open_dataset(Path(config["paths"]["geos_out"]) / config["inversion_constants"]["case"] / "combined_mf_tutorial.nc") as load:
    xr_data = load.load()

print(xr_data)

<xarray.Dataset>
Dimensions:        (obs_time: 12, site: 42)
Coordinates:
  * obs_time       (obs_time) datetime64[ns] 2010-01-31 ... 2010-12-31
  * site           (site) object 'altNOAAsurf' 'ascNOAAsurf' ... 'zepNOAAsurf'
Data variables: (12/30)
    obs_lat        (site, obs_time) float64 82.45 82.45 82.45 ... 78.91 78.91
    obs_lon        (site, obs_time) float64 -62.51 -62.51 -62.51 ... 11.89 11.89
    obs_alt        (site, obs_time) float64 190.0 190.0 190.0 ... 479.0 479.0
    obs_value      (site, obs_time) float64 323.3 323.5 323.6 ... 324.1 324.0
    obs_value_unc  (site, obs_time) float64 0.2779 0.26 0.3602 ... 0.26 0.3334
    CH4_R00        (site, obs_time) float32 4.308 4.304 4.305 ... 4.315 4.314
    ...             ...
    CH4_R19        (site, obs_time) float32 3.211 3.211 3.209 ... 3.217 3.219
    CH4_R20        (site, obs_time) float32 11.37 11.37 11.37 ... nan 11.4 11.4
    CH4_R21        (site, obs_time) float32 14.39 14.38 14.37 ... 14.42 14.42
    CH4_R22        (

This file contains information about the observations, as well as the model mole fraction. From the dimensions of this file, we can see that there are 12 observation times at 48 sites. Here's a quick summary of the variables:

- "obs_lat" - the latitude of the observations
- "obs_lon" - the longitude of the observations
- "obs_alt" - the altitude of the observations
- "obs_value" - the value of the observations
- "obs_value_unc" - the measurement uncertainty in the observations
- "CH4_R??" - The model mole fraction matching the observations for each tracer (one tracer for each transocm region)
- "obspack_id" - the unique identifier for each observation
- "CH4_sum" - The sum of the "CH4_R??" (only calculated for the base run)

## 5. Make the control mole fraction intermediate

The first argument is the name of the base run directory within the output directory, the second argument is the name of the mole fraction output, and the third argument is the name of the control mole fraction intermeidate that is output.

In [15]:
subprocess.call(f'Rscript ../intermediates/control-mole-fraction.R --case "{config["inversion_constants"]["case"]}" --mf-file "combined_mf_tutorial" --output "control-mole-fraction-tutorial"', shell = True)



Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

here() starts at /user/home/as16992/global_n2o_inversion
Warning message:
replacing previous import ‘Matrix::update’ by ‘stats::update’ when loading ‘wombat’ 


0

Look at the control mole fraction intermediate:

In [16]:
%%R

control_mf <- fst::read_fst(sprintf("%s/%s", config$paths$geos_inte, "control-mole-fraction-tutorial.fst"))

head(control_mf)

                                                                                                         observation_id
1 obspack_multi-species_1_CCGGSurfaceFlask_v2.0_2021-02-09~n2o_altNOAAsurf_surface-flask_1_ccgg_Event~altNOAAsurf201001
2 obspack_multi-species_1_CCGGSurfaceFlask_v2.0_2021-02-09~n2o_altNOAAsurf_surface-flask_1_ccgg_Event~altNOAAsurf201002
3 obspack_multi-species_1_CCGGSurfaceFlask_v2.0_2021-02-09~n2o_altNOAAsurf_surface-flask_1_ccgg_Event~altNOAAsurf201003
4 obspack_multi-species_1_CCGGSurfaceFlask_v2.0_2021-02-09~n2o_altNOAAsurf_surface-flask_1_ccgg_Event~altNOAAsurf201004
5 obspack_multi-species_1_CCGGSurfaceFlask_v2.0_2021-02-09~n2o_altNOAAsurf_surface-flask_1_ccgg_Event~altNOAAsurf201005
6 obspack_multi-species_1_CCGGSurfaceFlask_v2.0_2021-02-09~n2o_altNOAAsurf_surface-flask_1_ccgg_Event~altNOAAsurf201006
  observation_type resolution       time latitude longitude      co2 model_id
1          obspack    obspack 2010-01-31  82.4508  -62.5072 322.5403        1
2   

### Variables

- "observation_id" identifies the observation uniquely. In the original WOMBAT this was just the obspack ID, I have adapted mine so there is a station, network, and date identifier at the end.

- "observation_type" identifies the type of observation. In the original WOMBAT this was used to differentiate between LN and LG modes in satellite data etc, mine are all obspack

- "resolution" gives the resolution of the observation, i.e. in the original WOMBAT there was hourly, daily, and obspack data. In mine there is just obspack data combined to monthly means

- "time" identifies the time of the observation

- "latitude" is the latitude of the observation in degrees

- "longitude" is the longitude of the observation in degrees

- "co2" is the mole fraction of your gas of interest. The CO2 label is a hangover from original WOMBAT

- "model_id" identifies the grid cell and the month

### Ordering

Ordered by model_id. Must be ordered in the same way as observations.fst!

## 6. Make the sensitivities intermediate

This script takes the mole fraction file (combined_mf_tutorial.nc), and the control mole fraction intermediate (control-mole-fraction-tutorial.fst), and outputs the sensitivity intermediate (sensitivities_tutorial.fst).

In [17]:
subprocess.call('Rscript ../intermediates/sensitivities.R --mf-file "combined_mf_tutorial.nc" --control-mf "control-mole-fraction-tutorial.fst" --output "sensitivities_tutorial.fst"', shell = True)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

here() starts at /user/home/as16992/global_n2o_inversion

Attaching package: ‘lubridate’

The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union



[1] "Running main code of sensitivities script..."
[1] 2010
[1] 2010
[1] "/user/work/as16992/geoschem/N2O/output/201001/combined_mf_tutorial.nc"
[1] "/user/work/as16992/geoschem/N2O/output/201002/combined_mf_tutorial.nc"
[1] "/user/work/as16992/geoschem/N2O/output/201003/combined_mf_tutorial.nc"
[1] "/user/work/as16992/geoschem/N2O/output/201004/combined_mf_tutorial.nc"
[1] "/user/work/as16992/geoschem/N2O/output/201005/combined_mf_tutorial.nc"
[1] "/user/work/as16992/geoschem/N2O/output/201006/combined_mf_tutorial.nc"
[1] "/user/work/as16992/geoschem/N2O/output/201007/combined_mf_tutorial.nc"
[1] "/user/work/as16992/geoschem/N2O/output/201008/combined_mf_tutorial.nc"
[1] "/user/work/as16992/geoschem/N2O/output/201009/combined_mf_tutorial.nc"
[1] "/user/work/as16992/geoschem/N2O/output/201010/combined_mf_tutorial.nc"
[1] "/user/work/as16992/geoschem/N2O/output/201011/combined_mf_tutorial.nc"
[1] "/user/work/as16992/geoschem/N2O/output/201012/combined_mf_tutorial.nc"


0

Look at the sensitivities intermediate:

In [18]:
%%R

sensitivities <- fst::read_fst(sprintf("%s/%s", config$paths$geos_inte, "sensitivities_tutorial.fst"))

head(sensitivities)

  region from_month_start model_id resolution co2_sensitivity
1      0       2010-01-01        1    obspack     0.004066706
2      0       2010-01-01        2    obspack     0.012047052
3      0       2010-01-01        3    obspack     0.010704041
4      0       2010-01-01        4    obspack     0.008080006
5      0       2010-01-01        5    obspack     0.007016897
6      0       2010-01-01        6    obspack     0.006349087


### Variables

- "region" identifies the TRANSCOM region of the perturbation

- "from_month_start" identifies the time that the emissions were perturbed

- "model_id" identifies the grid cell and the month

- "resolution" gives the resolution of the observation, i.e. in the original WOMBAT there was hourly, daily, and obspack data. In mine there is just obspack data combined to monthly means

- "co2_sensitivity" is the difference in mole fraction between the perturbed and the base run for that grid cell

### Ordering

Ordered by region, then from_month_start, then model_id, then resolution

## 7. Make the observations intermediate

This script extracts the observations from the mole fraction file (combined_mf_tutorial.nc) and saves it as the observations intermediate (observations_tutorial.fst).

In [19]:
subprocess.call('Rscript ../intermediates/observations.R --mf-file "combined_mf_tutorial.nc" --model-err "" --output "observations_tutorial.fst"', shell = True)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

here() starts at /user/home/as16992/global_n2o_inversion
Warning message:
replacing previous import ‘Matrix::update’ by ‘stats::update’ when loading ‘wombat’ 


0

Look at the observations intermediate:

In [20]:
%%R

observations <- fst::read_fst(sprintf("%s/%s", config$paths$geos_inte, "observations_tutorial.fst"))

head(observations)

                                                                                                         observation_id
1 obspack_multi-species_1_CCGGSurfaceFlask_v2.0_2021-02-09~n2o_altNOAAsurf_surface-flask_1_ccgg_Event~altNOAAsurf201001
2 obspack_multi-species_1_CCGGSurfaceFlask_v2.0_2021-02-09~n2o_altNOAAsurf_surface-flask_1_ccgg_Event~altNOAAsurf201002
3 obspack_multi-species_1_CCGGSurfaceFlask_v2.0_2021-02-09~n2o_altNOAAsurf_surface-flask_1_ccgg_Event~altNOAAsurf201003
4 obspack_multi-species_1_CCGGSurfaceFlask_v2.0_2021-02-09~n2o_altNOAAsurf_surface-flask_1_ccgg_Event~altNOAAsurf201004
5 obspack_multi-species_1_CCGGSurfaceFlask_v2.0_2021-02-09~n2o_altNOAAsurf_surface-flask_1_ccgg_Event~altNOAAsurf201005
6 obspack_multi-species_1_CCGGSurfaceFlask_v2.0_2021-02-09~n2o_altNOAAsurf_surface-flask_1_ccgg_Event~altNOAAsurf201006
  observation_type       time longitude latitude      co2 co2_error
1          obspack 2010-01-31  -62.5072  82.4508 323.3229 0.2778638
2          obspack 2010-

### Variables

- "observation_id" identifies the observation uniquely. In the original WOMBAT this was just the obspack ID, I have adapted mine so there is a station, network, and date identifier at the end.

- "observation_type" identifies the type of observation. In the original WOMBAT this was used to differentiate between LN and LG modes in satellite data etc, mine are all obspack

- "time" identifies the time of the observation

- "longitude" is the longitude of the observation in degrees

- "latitude" is the latitude of the observation in degrees

- "co2" is the mole fraction of your gas of interest. The CO2 label is a hangover from original WOMBAT

- "co2_error" is the error in the measurement of your "co2" value

- "oco2_airmass", "oco2_co2_grad_del", "oco2_log_dws", "oco2_dp", "oco2_s31", "oco2_operation_mode", "tccon_station_id" are hangovers from original WOMBAT, mine are all NA

- "altitude" is the altitude of the observation in m

- "overall_observation_mode" is a hangover from original WOMBAT, they had LN, LG etc, I only have IS

- "observation_group" is the first bit of the obspack_id, contains info on the gas, network, site, type of measurement. This is what decides the grouping for the gamma parameter (error inflation factors) in WOMBAT

- "obspack_site" is the site for the obspack observations

- "obspack_site_type" is the type of site, mine are all surface

- "obspack_measurement_type" is the type of measurement, mine are all flask


### Ordering

Ordered by observation_id. Must be ordered in the same way as control-mole-fraction.fst!

## 8. Make the process model

In [21]:
my_env = os.environ.copy()
my_env["INVERSION_BASE_PARTIAL"] = f'{config["paths"]["wombat_paper"]}/3_inversion/src/partials/base.R'
subprocess.call(f'Rscript ../intermediates/process-model.R \
--control-emissions {config["paths"]["geos_inte"]}/control-emissions-tutorial.fst \
--perturbations {config["paths"]["geos_inte"]}/perturbations_tutorial.fst \
--control-mole-fraction {config["paths"]["geos_inte"]}/control-mole-fraction-tutorial.fst \
--sensitivities {config["paths"]["geos_inte"]}/sensitivities_tutorial.fst \
--output {config["paths"]["geos_inte"]}/process-model-tutorial.rds', shell = True, env = my_env)

Warning message:
replacing previous import ‘Matrix::update’ by ‘stats::update’ when loading ‘wombat’ 
INFO [2022-02-15 11:01:35] Loading control emissions
INFO [2022-02-15 11:01:35] Loading perturbations
INFO [2022-02-15 11:01:36] Loading control mole fraction
INFO [2022-02-15 11:01:36] Loading sensitivities
INFO [2022-02-15 11:01:36] Constructing process model
INFO [2022-02-15 11:01:38] Saving
INFO [2022-02-15 11:01:39] Done


0

Read in the process model and have a look at its contents:

In [22]:
%%R
process_model <- readRDS(paste0(config$paths$geos_inte, "/process-model-tutorial.rds"))

names(process_model)

 [1] "control_emissions"     "control_mole_fraction" "perturbations"        
 [4] "lag"                   "H"                     "regions"              
 [7] "Gamma"                 "kappa"                 "kappa_prior_mean"     
[10] "kappa_prior_variance"  "a_prior"               "a_factor"             
[13] "w_prior"               "w_factor"              "Psi"                  
[16] "eta_prior_mean"        "eta_prior_precision"  


This contains a lot of information! Beware, the original WOMBAT paper and the code are often not consistent, so here are the code process model parameters I cared about:
- "a" - called kappa in the paper, these are the autocorrelations on the AR prior for the alphas. If this is omitted it's assumed unknown. Can also be included with NA entries, then the NA entries are assumed unknown.
- "w" - called tau_w in the paper. If omitted, assumed unknown. If provided, NA entries are assumed unknown. In a further difference to the paper, w is not the diagonal of the precision matrix Q_w, but is the diagonal of the precision matrix Q_alpha (alpha are the flux scaling factors). Therefore, changing w_prior is how the prior on the flux scaling factors is set.
These two parameters are not in the process_model created above, because they are unknown and will be solved for in the inversion.

Now we can make sense of the parameters in the process model:
- "control_emissions" - as above
- "control_mole_fraction" - as above
- "perturbations" - as above       
- "lag" - how many months to truncate the mole-fraction basis functions to. Not used in the original WOMBAT paper or mine; can be set to infinity
- "H" - transport matrix. Can be provided manually, otherwise it will be constructed from control_mole_fraction and sensitivities                    
- "regions" - a vector of the unique values from the perturbations region column             
- "Gamma" - not in the original WOMBAT paper or mine, can ignore (Gamma times kappa is the prior mean for the alphas; by default kappa = 0, so the prior mean is zero)                 
- "kappa" - not in the original WOMBAT paper or mine, different to kappa in the paper, (Mike says sorry). Can ignore (see above)                
- "kappa_prior_mean" - not in the original WOMBAT paper or mine. Prior on kappa, can ignore if kappa = 0 as is the default  
- "kappa_prior_variance" - not in the original WOMBAT paper or mine. Prior on kappa, can ignore if kappa = 0 as is the default 
- "a_prior" - the beta prior for a (mine is list(shape=1, rate=1), aka a uniform distribution between 0 and 1)              
- "a_factor" - you can share the a parameters between regions using this factor; by default, one per region            
- "w_prior" - the gamma prior for w (mine is list(shape=4, rate=0.7))             
- "w_factor" - like a_factor, you can share w's between regions             
- "Psi" - not in the original WOMBAT paper or mine. Lets you specify extra basis functions that aren't linked to flux.  
- "eta" -  not in the original WOMBAT paper or mine. Eta are coefficients for Psi. Assumed unknown if omitted.               
- "eta_prior_mean" - not in the original WOMBAT paper or mine. Prior for eta
- "eta_prior_precision" - not in the original WOMBAT paper or mine. Prior for eta 


Use the box below as an example of how to access the parts of the process_model:

In [23]:
%%R
head(process_model$control_emissions)


  month_start model_id       area  type flux_density longitude cell_width
1  2010-01-01        1 2157765258  land 1.010600e-14      -180          5
2  2010-01-01        1 2157765258 ocean 0.000000e+00      -180          5
3  2010-01-01        2 2157765258  land 9.989040e-15      -175          5
4  2010-01-01        2 2157765258 ocean 0.000000e+00      -175          5
5  2010-01-01        3 2157765258  land 9.757725e-15      -170          5
6  2010-01-01        3 2157765258 ocean 0.000000e+00      -170          5
  latitude cell_height
1      -89           2
2      -89           2
3      -89           2
4      -89           2
5      -89           2
6      -89           2


## 9. Make the measurement model
There are four not obvious arguments that can be passed to my measurement-model.R, which relate to the gamma parameters (1 / gamma described in the original WOMBAT paper, a bit more about this in the next box). The four parameters are:
- "gamma-prior-min" - 5% percentile of the gamma prior for gamma
- "gamma-prior-max" - 95% percentile of the gamma prior for gamma
- "gamma-prior-shape" - shape of the prior gamma distribution for gamma
- "gamma-prior-rate" - rate of the prior gamma distribution for gamma

set either "gamma-prior-min" and "gamma-prior-max", OR "gamma-prior-shape" and "gamma-prior-rate". This was my hacky way to force gamma to be 1, because for some reason, just setting gamma to 1 produces an error (Michael says it shouldn't but it does).

In the case demonstrated below, the 5% and 95% percentile of the prior are 0.1 and 1.9.  So it says that it's reasonable that the measurement precision (variance) is between 10x lower (variance 10x higher) and 1.9x higher (variance 1.9x lower) than the given error. Our reasoning was that it's better to err on allowing the measurements to be considered worse than stated rather than better than stated.

In the case where I force gamma to be basically fixed at 1, I used gamma-prior-shape of 100000000 and gamma-prior-rate of 0.00000001.

In [24]:
subprocess.call(f'Rscript ../intermediates/measurement-model.R \
--observations {config["paths"]["geos_inte"]}/observations_tutorial.fst \
--process-model {config["paths"]["geos_inte"]}/process-model-tutorial.rds \
--gamma-prior-min 0.1 \
--gamma-prior-max 1.9 \
--output {config["paths"]["geos_inte"]}/measurement-model-tutorial.rds', shell = True, env = my_env)

Warning message:
replacing previous import ‘Matrix::update’ by ‘stats::update’ when loading ‘wombat’ 
INFO [2022-02-15 11:02:17] Loading observations
INFO [2022-02-15 11:02:17] Loading process model


[1] "using min and max"


INFO [2022-02-15 11:02:19] Constructing bias matrix
INFO [2022-02-15 11:02:19] Constructing measurement model
INFO [2022-02-15 11:02:19] Saving
INFO [2022-02-15 11:02:19] Done


0

In [25]:
%%R
measurement_model <- readRDS(paste0(config$paths$geos_inte, "/measurement-model-tutorial.rds"))

names(measurement_model)

 [1] "observations"          "C"                     "time"                 
 [4] "measurement_variance"  "A"                     "beta_prior_mean"      
 [7] "beta_prior_precision"  "attenuation_variables" "attenuation_factor"   
[10] "gamma_prior"           "rho_prior"             "ell_prior"            


Measurement model parameters to be solved for in the original WOMBAT paper:
- "beta" - same as paper, coefficients for A. If omitted, assumed unknown
- "rho" - same as paper. Assumed unknown if omitted. If you set this to 0, the data are assumed uncorrelated, no inference is done for ell.
- "ell" - same as paper, assumed unknown if omitted

A key difference from the original WOMBAT paper:
- "gamma" - attenuation factors, defined as 1 / inflation factor. Slightly different to the paper, it's 1 / gamma for the gamma in the paper because we actually scale the precisions not the variances. If omitted, assumed unknown (the default)


The contents of my measurement model output as shown above:
- "observations" - as above
- "C" - matrix that maps control_mole_fraction to observations; build from matching variable if C is not provided
- "time" - the time variable, vector of observation times, should be POSIXct or similar
- "measurement_variance" - exactly what it says
- "A" - same as paper, design matrix for biases
- "attenuation_variables" - name of variable to use in observations data frame to assign inflation factors (1 / gammas) to observations. Basically used for different observation modes/sites
- "attenuation_factor" - alternative to attenuation_variables, you can specify the mapping directly with a factor.
- "beta_prior_mean" / "beta_prior_variance" / "beta_prior_precision" - prior for beta
- "gamma_prior" - prior for gamma. It's a gamma distribution, matching the inverse gamma in the paper (see note above for parameterisation)
- "rho_prior" - prior for rho, a uniform distribution
- "ell_prior" - prior for ell, a gamma distribution. The unit specifies how the time variable (as POSIXct) is converted into a number for the purposes of calculating the model-data mismatch correlation.

Some variables that can be passed to flux_measurement_model but are not present in my output:
- "biases" - formula for biases
- "matching" - sets up the matching between the observations data frame and the control_mole_fraction data frame in the measurement model. Should be a vector of integers, one for each observations, specifying the row number of the corresponding row in control_mole_fraction. Can be omitted if C is provided
- "process_model" - as above


Use the box below as an example of how to access the parts of the process_model:

In [26]:
%%R
head(measurement_model$observations)


                                                                                                         observation_id
1 obspack_multi-species_1_CCGGSurfaceFlask_v2.0_2021-02-09~n2o_altNOAAsurf_surface-flask_1_ccgg_Event~altNOAAsurf201001
2 obspack_multi-species_1_CCGGSurfaceFlask_v2.0_2021-02-09~n2o_altNOAAsurf_surface-flask_1_ccgg_Event~altNOAAsurf201002
3 obspack_multi-species_1_CCGGSurfaceFlask_v2.0_2021-02-09~n2o_altNOAAsurf_surface-flask_1_ccgg_Event~altNOAAsurf201003
4 obspack_multi-species_1_CCGGSurfaceFlask_v2.0_2021-02-09~n2o_altNOAAsurf_surface-flask_1_ccgg_Event~altNOAAsurf201004
5 obspack_multi-species_1_CCGGSurfaceFlask_v2.0_2021-02-09~n2o_altNOAAsurf_surface-flask_1_ccgg_Event~altNOAAsurf201005
6 obspack_multi-species_1_CCGGSurfaceFlask_v2.0_2021-02-09~n2o_altNOAAsurf_surface-flask_1_ccgg_Event~altNOAAsurf201006
  observation_type       time longitude latitude      co2 co2_error
1          obspack 2010-01-31  -62.5072  82.4508 323.3229 0.2778638
2          obspack 2010-

## 10. Make the real model

This code combines the process and measurement model, making sure it is set up for the correct model case, e.g. my model case ("IS-RHO0-FIXEDA-VARYW-NOBIAS") is composed of five parts:
- "IS" - only use observations with "overall_observation_mode" == "IS"
- "RHO0" - sets "rho" to 0 and "ell" to 1, this turns off the correlation within WOMBAT
- "FIXEDA" - sets all "a" to 0, won't be solved for in the inversion
- "VARYW" - leaves "w" as unknown so that all are solved for in the inversion
- "NOBIAS" - turns off the measurement bias described in the original WOMBAT paper by setting A and the beta prior to zeros

There are currently limited options coded in, though it is easy to add your own in real-model-case.R. Here are the other current options:
- "FIXEDAO" - sets "a" for oceans to 0, "a" for land to NA, so only "a" for land is solved for in the inversion 
- "FIXEDW" - sets all "w" to 4, won't be solved for in the inversion
- "FIXEDWO5" - sets "w" for oceans to 4, "w" for land to NA, so only "w" for land is solved for in the inversion 

This just means you only need to store one process and measurement model, which can then be combined to slightly different inversion parameterisations. It is this real model that is used in the WOMBAT inversion, (although it is currently designed to read in H and the sensitivities from the process model instead for some reason I don't understand).

In [27]:
subprocess.call(f'Rscript ../intermediates/real-model-case.R \
--case IS-RHO0-FIXEDA-VARYW-NOBIAS \
--measurement-model {config["paths"]["geos_inte"]}/measurement-model-tutorial.rds \
--process-model {config["paths"]["geos_inte"]}/process-model-tutorial.rds \
--output {config["paths"]["geos_inte"]}/real-model-IS-RHO0-FIXEDA-VARYW-NOBIAS-tutorial.rds', shell = True, env = my_env)

Warning message:
replacing previous import ‘Matrix::update’ by ‘stats::update’ when loading ‘wombat’ 
here() starts at /user/home/as16992/global_n2o_inversion
INFO [2022-02-15 11:02:40] [IS-RHO0-VARYA-VARYW-NOBIAS] Loading measurement model
INFO [2022-02-15 11:02:40] [IS-RHO0-VARYA-VARYW-NOBIAS] Loading process model
INFO [2022-02-15 11:02:42] [IS-RHO0-VARYA-VARYW-NOBIAS] Constructing case
INFO [2022-02-15 11:02:42] [IS-RHO0-VARYA-VARYW-NOBIAS] Saving
INFO [2022-02-15 11:02:42] [IS-RHO0-VARYA-VARYW-NOBIAS] Done


0

In [28]:
%%R
real_model <- readRDS(sprintf("%s/real-model-IS-RHO0-FIXEDA-VARYW-NOBIAS-tutorial.rds", config$paths$geos_inte))

names(real_model)

[1] "process_model"     "measurement_model"


You can see the differences between the measurement model and the real model:

In [29]:
%%R
measurement_model$beta_prior_precision

R[write to console]: Loading required package: Matrix



42 x 42 diagonal matrix of class "ddiMatrix"
      [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11] [,12] [,13]
 [1,] 0.01    .    .    .    .    .    .    .    .     .     .     .     .
 [2,]    . 0.01    .    .    .    .    .    .    .     .     .     .     .
 [3,]    .    . 0.01    .    .    .    .    .    .     .     .     .     .
 [4,]    .    .    . 0.01    .    .    .    .    .     .     .     .     .
 [5,]    .    .    .    . 0.01    .    .    .    .     .     .     .     .
 [6,]    .    .    .    .    . 0.01    .    .    .     .     .     .     .
 [7,]    .    .    .    .    .    . 0.01    .    .     .     .     .     .
 [8,]    .    .    .    .    .    .    . 0.01    .     .     .     .     .
 [9,]    .    .    .    .    .    .    .    . 0.01     .     .     .     .
[10,]    .    .    .    .    .    .    .    .    .  0.01     .     .     .
[11,]    .    .    .    .    .    .    .    .    .     .  0.01     .     .
[12,]    .    .    .    .    .    .    .    .    .     

In [30]:
%%R
real_model$measurement_model$beta_prior_precision

0 x 0 diagonal matrix of class "ddiMatrix"
<0 x 0 matrix>


Congratulations, you are now ready to run the WOMBAT inversion!